In [3]:
# from libraries
import pandas as pd
import dask.dataframe as dd
# from files
import params
import auth
import get_data_util
import get_sp500_util as sp500
import get_dow_jones_util as dj

In [6]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
import pandas as pd
import dask.dataframe as dd
from io import StringIO
import quandl
import time
import boto3
import datetime

# from files:
import auth
import params

In [79]:
ticker_list = dj.get_dow_jones(url=dj.url, headers=auth.user_agent)
quandl_parameters = [a for a in dir(params) if ("_parameters" in a)]

Process Complete!!!


In [37]:
stock_info_list = []
alpha_vantage_base = "https://www.alphavantage.co"  # url base
param_dict_names = [name for name in dir(params) if ('_params' in name)]
param_dict_list = [getattr(params, name) for name in param_dict_names]

In [38]:
param_dict_names

['ad_params',
 'aroon_params',
 'bbands_close_params',
 'bbands_high_params',
 'bbands_low_params',
 'bbands_open_params',
 'cci_params',
 'ema_close_params',
 'ema_high_params',
 'ema_low_params',
 'ema_open_params_200',
 'macd_close_params',
 'macd_high_params',
 'macd_low_params',
 'macd_open_params',
 'obv_params',
 'rsi_close_params',
 'rsi_high_params',
 'rsi_low_params',
 'rsi_open_params',
 'stoch_params',
 'stock_time_series_adjusted_params']

In [75]:
for stock_symbol in ticker_list[:1]:
    stock_indicators = []
    for i in range(len(param_dict_list)):
        session = requests.Session()
        retry = Retry(connect=5, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount("http://", adapter)
        session.mount("https://", adapter)
        param_dict_list[i]["symbol"] = stock_symbol
        r = session.get(alpha_vantage_base + '/query', params=param_dict_list[i], headers=auth.user_agent)
        df = pd.read_csv(StringIO(r.content.decode('utf-8')), engine="python").loc[1:,:]
        df.columns = [param_dict_list[i]["function"] + "_" + str(column) for column in df.columns]
        df.set_index(df.columns[0], inplace=True)
        if param_dict_list[i]["function"] == "TIME_SERIES_DAILY_ADJUSTED":
            df["stock_symbol"] = stock_symbol
        stock_indicators.append(df)
        print("Successfully extracted {}!".format(stock_symbol+"_"+param_dict_names[i].replace("_params","")))
    stock_info = pd.concat(stock_indicators, axis=1)
    stock_info = stock_info.loc[:,~stock_info.columns.duplicated()]
#    stock_info_list.append(stock_info)
#concat_data = pd.concat(stock_info_list, axis=0, sort=False, ignore_index=True)

Successfully extracted MMM_ad!
Successfully extracted MMM_aroon!
Successfully extracted MMM_bbands_close!
Successfully extracted MMM_bbands_high!
Successfully extracted MMM_bbands_low!
Successfully extracted MMM_bbands_open!
Successfully extracted MMM_cci!
Successfully extracted MMM_ema_close!
Successfully extracted MMM_ema_high!
Successfully extracted MMM_ema_low!
Successfully extracted MMM_ema_open_200!
Successfully extracted MMM_macd_close!
Successfully extracted MMM_macd_high!
Successfully extracted MMM_macd_low!
Successfully extracted MMM_macd_open!
Successfully extracted MMM_obv!
Successfully extracted MMM_rsi_close!
Successfully extracted MMM_rsi_high!
Successfully extracted MMM_rsi_low!
Successfully extracted MMM_rsi_open!
Successfully extracted MMM_stoch!
Successfully extracted MMM_stock_time_series_adjusted!


D:\Anaconda_Navigator\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [69]:
len(stock_indicators)

22

In [77]:
stock_info

,AD_Chaikin A/D,AROON_Aroon Down,AROON_Aroon Up,BBANDS_Real Lower Band,BBANDS_Real Middle Band,BBANDS_Real Upper Band,CCI_CCI,EMA_EMA,MACD_MACD,MACD_MACD_Hist,...,STOCH_SlowK,TIME_SERIES_DAILY_ADJUSTED_open,TIME_SERIES_DAILY_ADJUSTED_high,TIME_SERIES_DAILY_ADJUSTED_low,TIME_SERIES_DAILY_ADJUSTED_close,TIME_SERIES_DAILY_ADJUSTED_adjusted_close,TIME_SERIES_DAILY_ADJUSTED_volume,TIME_SERIES_DAILY_ADJUSTED_dividend_amount,TIME_SERIES_DAILY_ADJUSTED_split_coefficient,stock_symbol
1999-09-13,7.436056e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,96.7500,99.1369,96.7500,98.500,27.6777,1594600,0.00,1.0,MMM
1999-09-14,-9.270412e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,98.2582,99.0000,96.8130,97.063,27.2738,2165800,0.00,1.0,MMM
1999-09-15,-1.005208e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.6250,98.5630,96.7498,97.625,27.4318,2256800,0.00,1.0,MMM
1999-09-16,-1.904488e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.6250,97.6250,95.7500,96.063,26.9928,1350000,0.00,1.0,MMM
1999-09-17,-1.164039e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.1250,99.1250,96.6880,98.250,27.6074,2626600,0.00,1.0,MMM
1999-09-20,-1.716979e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.7467,99.3130,97.7500,98.188,27.5899,1258000,0.00,1.0,MMM
1999-09-21,-2.120563e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.2500,97.7500,95.7500,96.563,27.1333,2158200,0.00,1.0,MMM
1999-09-22,-2.412798e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,96.5630,96.8750,94.7500,95.625,26.8698,1656000,0.00,1.0,MMM
1999-09-23,-2.062682e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.4263,95.7500,96.2500,93.8750,95.250,26.7645,2217400,0.00,1.0,MMM
1999-09-24,-3.426121e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.6470,94.0000,94.0630,91.1181,91.813,25.7985,2582000,0.00,1.0,MMM


In [74]:
len(stock_info.columns)

34

In [28]:
list(stock_info_list[0].columns)

['AD_Chaikin A/D',
 'AROON_Aroon Down',
 'AROON_Aroon Up',
 'BBANDS_Real Lower Band',
 'BBANDS_Real Middle Band',
 'BBANDS_Real Upper Band',
 'BBANDS_Real Lower Band',
 'BBANDS_Real Middle Band',
 'BBANDS_Real Upper Band',
 'BBANDS_Real Lower Band',
 'BBANDS_Real Middle Band',
 'BBANDS_Real Upper Band',
 'BBANDS_Real Lower Band',
 'BBANDS_Real Middle Band',
 'BBANDS_Real Upper Band',
 'CCI_CCI',
 'EMA_EMA',
 'EMA_EMA',
 'EMA_EMA',
 'EMA_EMA',
 'MACD_MACD',
 'MACD_MACD_Hist',
 'MACD_MACD_Signal',
 'MACD_MACD',
 'MACD_MACD_Hist',
 'MACD_MACD_Signal',
 'MACD_MACD',
 'MACD_MACD_Hist',
 'MACD_MACD_Signal',
 'MACD_MACD',
 'MACD_MACD_Hist',
 'MACD_MACD_Signal',
 'OBV_OBV',
 'RSI_RSI',
 'RSI_RSI',
 'RSI_RSI',
 'RSI_RSI',
 'STOCH_SlowD',
 'STOCH_SlowK',
 'TIME_SERIES_DAILY_ADJUSTED_open',
 'TIME_SERIES_DAILY_ADJUSTED_high',
 'TIME_SERIES_DAILY_ADJUSTED_low',
 'TIME_SERIES_DAILY_ADJUSTED_close',
 'TIME_SERIES_DAILY_ADJUSTED_adjusted_close',
 'TIME_SERIES_DAILY_ADJUSTED_volume',
 'TIME_SERIES_DAI

In [27]:
list(stock_info_list[2].columns)

['AD_Chaikin A/D',
 'AROON_Aroon Down',
 'AROON_Aroon Up',
 'BBANDS_Real Lower Band',
 'BBANDS_Real Middle Band',
 'BBANDS_Real Upper Band',
 'BBANDS_Real Lower Band',
 'BBANDS_Real Middle Band',
 'BBANDS_Real Upper Band',
 'BBANDS_Real Lower Band',
 'BBANDS_Real Middle Band',
 'BBANDS_Real Upper Band',
 'BBANDS_Real Lower Band',
 'BBANDS_Real Middle Band',
 'BBANDS_Real Upper Band',
 'CCI_CCI',
 'EMA_EMA',
 'EMA_EMA',
 'EMA_EMA',
 'EMA_EMA',
 'MACD_MACD',
 'MACD_MACD_Hist',
 'MACD_MACD_Signal',
 'MACD_MACD',
 'MACD_MACD_Hist',
 'MACD_MACD_Signal',
 'MACD_MACD',
 'MACD_MACD_Hist',
 'MACD_MACD_Signal',
 'MACD_MACD',
 'MACD_MACD_Hist',
 'MACD_MACD_Signal',
 'OBV_OBV',
 'RSI_RSI',
 'RSI_RSI',
 'RSI_RSI',
 'RSI_RSI',
 'STOCH_SlowD',
 'STOCH_SlowK',
 'TIME_SERIES_DAILY_ADJUSTED_open',
 'TIME_SERIES_DAILY_ADJUSTED_high',
 'TIME_SERIES_DAILY_ADJUSTED_low',
 'TIME_SERIES_DAILY_ADJUSTED_close',
 'TIME_SERIES_DAILY_ADJUSTED_adjusted_close',
 'TIME_SERIES_DAILY_ADJUSTED_volume',
 'TIME_SERIES_DAI

In [81]:
quandl.ApiConfig.api_key = auth.apikey_quandl
indicator_data_list = []

In [83]:
for param in quandl_parameters:
        dict_ = getattr(params, param)
        data = quandl.get("{}/{}".format(dict_["database_code"], dict_["dataset_code"]),
                          start_date=dict_["start_date"], end_date=dict_["end_date"])
        data.columns = [param.split("_")[0] + "_" + str(column) for column in data.columns]
        indicator_data_list.append(data)
indicator_data = pd.concat(indicator_data_list, axis=1, sort=False)

In [84]:
indicator_data

,corporatebond_BAMLC0A0CMEY,crudeoil_Open,crudeoil_High,crudeoil_Low,crudeoil_Last,crudeoil_Change,crudeoil_Settle,crudeoil_Volume,crudeoil_Previous Day Open Interest,dollar_Open,...,sp500_Last,sp500_Change,sp500_Settle,sp500_Volume,sp500_Previous Day Open Interest,treasury_5 YR,treasury_7 YR,treasury_10 YR,treasury_20 YR,treasury_30 YR
1999-09-13,7.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.550,...,1347.0,NaN,1347.0,69989.0,139221.0,NaN,NaN,NaN,NaN,NaN
1999-09-14,7.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.970,...,1337.6,NaN,1337.6,66668.0,105915.0,NaN,NaN,NaN,NaN,NaN
1999-09-15,7.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.340,...,1317.2,NaN,1317.2,44552.0,90258.0,NaN,NaN,NaN,NaN,NaN
1999-09-16,7.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.910,...,1319.2,NaN,1319.2,49522.0,90258.0,NaN,NaN,NaN,NaN,NaN
1999-09-17,7.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.170,...,1351.5,NaN,1351.5,82443.0,349363.0,NaN,NaN,NaN,NaN,NaN
1999-09-20,7.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.810,...,1348.8,NaN,1348.8,41713.0,347911.0,NaN,NaN,NaN,NaN,NaN
1999-09-21,7.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.910,...,1321.9,NaN,1321.9,100684.0,351841.0,NaN,NaN,NaN,NaN,NaN
1999-09-22,7.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.640,...,1321.8,NaN,1321.8,109084.0,351747.0,NaN,NaN,NaN,NaN,NaN
1999-09-23,7.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.010,...,1291.0,NaN,1291.0,111874.0,353707.0,NaN,NaN,NaN,NaN,NaN
1999-09-24,7.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.890,...,1288.5,NaN,1288.5,114307.0,354331.0,NaN,NaN,NaN,NaN,NaN


In [87]:
stock_info

,AD_Chaikin A/D,AROON_Aroon Down,AROON_Aroon Up,BBANDS_Real Lower Band,BBANDS_Real Middle Band,BBANDS_Real Upper Band,CCI_CCI,EMA_EMA,MACD_MACD,MACD_MACD_Hist,...,STOCH_SlowK,TIME_SERIES_DAILY_ADJUSTED_open,TIME_SERIES_DAILY_ADJUSTED_high,TIME_SERIES_DAILY_ADJUSTED_low,TIME_SERIES_DAILY_ADJUSTED_close,TIME_SERIES_DAILY_ADJUSTED_adjusted_close,TIME_SERIES_DAILY_ADJUSTED_volume,TIME_SERIES_DAILY_ADJUSTED_dividend_amount,TIME_SERIES_DAILY_ADJUSTED_split_coefficient,stock_symbol
1999-09-13,7.436056e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,96.7500,99.1369,96.7500,98.500,27.6777,1594600,0.00,1.0,MMM
1999-09-14,-9.270412e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,98.2582,99.0000,96.8130,97.063,27.2738,2165800,0.00,1.0,MMM
1999-09-15,-1.005208e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.6250,98.5630,96.7498,97.625,27.4318,2256800,0.00,1.0,MMM
1999-09-16,-1.904488e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.6250,97.6250,95.7500,96.063,26.9928,1350000,0.00,1.0,MMM
1999-09-17,-1.164039e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.1250,99.1250,96.6880,98.250,27.6074,2626600,0.00,1.0,MMM
1999-09-20,-1.716979e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.7467,99.3130,97.7500,98.188,27.5899,1258000,0.00,1.0,MMM
1999-09-21,-2.120563e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,97.2500,97.7500,95.7500,96.563,27.1333,2158200,0.00,1.0,MMM
1999-09-22,-2.412798e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,96.5630,96.8750,94.7500,95.625,26.8698,1656000,0.00,1.0,MMM
1999-09-23,-2.062682e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.4263,95.7500,96.2500,93.8750,95.250,26.7645,2217400,0.00,1.0,MMM
1999-09-24,-3.426121e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.6470,94.0000,94.0630,91.1181,91.813,25.7985,2582000,0.00,1.0,MMM


In [86]:
stock_info.join(indicator_data)

,AD_Chaikin A/D,AROON_Aroon Down,AROON_Aroon Up,BBANDS_Real Lower Band,BBANDS_Real Middle Band,BBANDS_Real Upper Band,CCI_CCI,EMA_EMA,MACD_MACD,MACD_MACD_Hist,...,sp500_Last,sp500_Change,sp500_Settle,sp500_Volume,sp500_Previous Day Open Interest,treasury_5 YR,treasury_7 YR,treasury_10 YR,treasury_20 YR,treasury_30 YR
1999-09-13,7.436056e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1347.0,NaN,1347.0,69989.0,139221.0,NaN,NaN,NaN,NaN,NaN
1999-09-14,-9.270412e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1337.6,NaN,1337.6,66668.0,105915.0,NaN,NaN,NaN,NaN,NaN
1999-09-15,-1.005208e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1317.2,NaN,1317.2,44552.0,90258.0,NaN,NaN,NaN,NaN,NaN
1999-09-16,-1.904488e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1319.2,NaN,1319.2,49522.0,90258.0,NaN,NaN,NaN,NaN,NaN
1999-09-17,-1.164039e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1351.5,NaN,1351.5,82443.0,349363.0,NaN,NaN,NaN,NaN,NaN
1999-09-20,-1.716979e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1348.8,NaN,1348.8,41713.0,347911.0,NaN,NaN,NaN,NaN,NaN
1999-09-21,-2.120563e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1321.9,NaN,1321.9,100684.0,351841.0,NaN,NaN,NaN,NaN,NaN
1999-09-22,-2.412798e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1321.8,NaN,1321.8,109084.0,351747.0,NaN,NaN,NaN,NaN,NaN
1999-09-23,-2.062682e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1291.0,NaN,1291.0,111874.0,353707.0,NaN,NaN,NaN,NaN,NaN
1999-09-24,-3.426121e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1288.5,NaN,1288.5,114307.0,354331.0,NaN,NaN,NaN,NaN,NaN
